In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
def precision(TP, FP): 
    return TP/(TP+FP)

def recall(TP, FN): 
    return TP/(TP+FN)

def F1(precision, recall):
    return 2/(1/precision + 1/recall)

In [52]:
def return_TP_FN_FP(keywords, ground_truth):
    keywords = set(keywords)
    ground_truth = set(ground_truth)
    TP = len(keywords.intersection(ground_truth) )
    FN = len(keywords.difference(ground_truth))
    FP = len(ground_truth.difference(keywords))
    return np.array([TP, FN, FP])

In [5]:
concepts_mapping = pd.read_csv('../data/ids_to_concept_mapping.csv')

In [7]:
taggs = pd.read_csv('../results/tagged_bertopic_bert_pretrained.csv')

In [37]:
ground_truth = pd.read_csv('../data/concepts_per_document.csv').drop('Concepts', axis=1)
ground_truth['Unique_concepts'] = ground_truth['Unique_concepts'].apply(lambda x: x.replace('\r\n', '').replace('\' \'','\',\'')).apply(eval)
ground_truth = ground_truth.explode('Unique_concepts')
assert max(ground_truth['Unique_concepts'].apply(lambda x: x.count('U'))) == 1
ground_truth['Unique_concepts'] = ground_truth['Unique_concepts'].apply(lambda x: x.replace('UMLS:', ''))
ground_truth.head()

,PMID,Unique_concepts
0,25763772,C4308010
0,25763772,C0854135
0,25763772,C0010674
0,25763772,C0024115
0,25763772,C3160731


In [38]:
merged = ground_truth.merge(concepts_mapping, left_on=['Unique_concepts'], right_on=['concept_id'], how='inner')

In [39]:
def collect_list(x): 
    result= {}
    result['ground_truth'] = list(x['concept_name'])
    return pd.Series(result, index=['ground_truth'])
ground_truth = merged.groupby('PMID').apply(collect_list).reset_index()

In [44]:
test_data = ground_truth.merge(taggs)

In [45]:
test_data.head(1)

,PMID,ground_truth,tagged_words
0,26868132,"[Population Groups, Persons, RBP4 protein, hum...","['Hyperemia', 'HN Protein', 'Intention', 'Insu..."


In [53]:
TP_FN_FP = np.zeros((3,))
for i, row in test_data.iterrows(): 
    TP_FN_FP += return_TP_FN_FP(eval(row['tagged_words']), row['ground_truth'])

In [56]:
precision_ = precision(TP_FN_FP[0], TP_FN_FP[2])
recall_ = recall(TP_FN_FP[0], TP_FN_FP[1])
precision_, recall_


(0.01000071947622131, 0.0158947970268725)

In [58]:
F1(precision_, recall_)

0.012276982865218158